In [25]:
import gensim, operator
from scipy import spatial
import numpy as np
from gensim.models import KeyedVectors

model_path = '/Users/pinky/Desktop/HUST/ANLY-610_project/ANLY-610/model/'
data_path= '/Users/pinky/Desktop/HUST/ANLY-610_project/data/'

In [26]:
import os
import json

import gensim, operator
from scipy import spatial
import numpy as np
from gensim.models import KeyedVectors

In [27]:
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model_word2vec = load_wordvec_model('Word2Vec', 'GoogleNews-vectors-negative300.bin', True)

Loading Word2Vec model...
Finished loading Word2Vec model...


In [36]:
def load_webhouse_data(data_name): 
    data_file = data_path + data_name + '.json'
    with open(data_file) as json_data:
        data = json.load(json_data)
        return data

In [37]:
data=load_webhouse_data('ANLY610_MSFT_Data')

In [38]:
import pandas as pd
import string
import pyspark
from pyspark.sql import SQLContext
from pyspark.ml.feature import CountVectorizer as CountVectorizer_pyspark
from pyspark.ml.feature import IDF 
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.mllib.clustering import LDA, LDAModel

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import LatentDirichletAllocation
import nltk, re
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary
import pyLDAvis.gensim

In [39]:
stopwords= set(nltk.corpus.stopwords.words('english'))

In [40]:
def tokenize_titles(title):
    tokens = nltk.word_tokenize(title)
    lmtzr = WordNetLemmatizer()
    filtered_tokens = []
    
    for token in tokens:
        token = token.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
        token = re.sub(r'[^a-zA-Z0-9 ]', '', token)
        if token not in stopwords:
            filtered_tokens.append(token.lower())
    
    lemmas = [lmtzr.lemmatize(t,'v') for t in filtered_tokens]

    return lemmas

In [33]:
def clstr_lda(num_topics, stories):
    # top words to be identified
    n_top_words = 10

    tf_vectorizer = CountVectorizer(max_df=100, min_df=2, max_features=500,
                                    tokenizer=tokenize_titles, ngram_range=(3,4))

    tf = tf_vectorizer.fit_transform(stories)

    lda = LatentDirichletAllocation(n_components=num_topics, max_iter=200,
                                    learning_method='batch', learning_offset=10.,
                                    random_state = 1)
    lda.fit(tf)
    tf_feature_names = tf_vectorizer.get_feature_names()

    # print top topic words
    topics = dict()
    for topic_idx, topic in enumerate(lda.components_):
        topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print("Topic #%d:" % topic_idx)
        print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
    return topics

In [34]:
feed_titles = []

for feed in data:
    feed_titles.append(str(feed['title']))

In [35]:
topics = clstr_lda(10, feed_titles)

Topic #0:
ibm  microsoft | ibm  microsoft  | 13  2019 | march 13  2019 | march 13  |  ibm  microsoft | 12  2019 | march 12  2019 | march 12  |  march 12
Topic #1:
 administrative assistant | sap se  |  sap se | bring xbox live |  sap se  | microsoft bring xbox | microsoft bring xbox live |  microsoft corporation | master chief collection pc | chief collection pc
Topic #2:
    | chief collection come | master chief collection come | chief collection come pc | collection come pc | halo  reach | windows 7 users | come pc  | march 14  | 14  2019
Topic #3:
music award 2019 | iheartradio music award 2019 | show project xcloud | pc game xbox | game xbox one | microsoft show project xcloud | microsoft show project | award 2019  | pc game xbox one | music award 2019 
Topic #4:
windows 10 automatically | windows 10 automatically uninstall | 10 automatically uninstall | android apps windows | project xcloud stream | apps windows 10 | android apps windows 10 | xcloud stream service | project xclou

In [41]:
titles = [[data[i]['title'], data[i]['published'][:10]]  for i in range(len(data))]
df_feeds = pd.DataFrame(titles,columns=['title', 'date'])
df_feeds.head()

,title,date
0,ICT and society portfolio section Essay,2019-03-12
1,Triggering plugins / workflows when associatin...,2019-03-11
2,Ways to Get a Free PVS-Studio License,2019-03-11
3,[UPDATE] MDB Viewer Plus v2.63,2019-03-12
4,Good news for software engineers as Microsoft ...,2019-03-12


In [42]:
df_daily_count = df_feeds.groupby('date')['title'].count()
df_daily_count.plot(kind='bar', x='date')

In [43]:
punctuation = set(string.punctuation) 
def text_cleanup(input_text):
    one = " ".join([i for i in input_text.lower().split() if i not in stopwords])
    two = "".join(re.sub(r'[^a-zA-Z ]', '', i) for i in one if i not in punctuation)
    three = [WordNetLemmatizer().lemmatize(i) for i in two.split()]
    return three

In [44]:
titles = df_feeds[['title']].applymap(text_cleanup)['title']
dictionary = Dictionary(titles)
dictionary.filter_extremes(no_below=10, no_above=0.8)
corpora = [dictionary.doc2bow(doc) for doc in titles]

# Running and Trainign LDA model on the document term matrix.
lda_model = ldamodel.LdaModel(corpora, num_topics=8, id2word = dictionary, passes=50)

In [45]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpora, dictionary)
vis

/Users/pinky/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.243717 -0.051218       1        1  20.439827
0      0.087814  0.133142       2        1  13.898444
6      0.204327 -0.005197       3        1  12.285803
1     -0.158683 -0.136201       4        1  12.085470
5     -0.160781  0.254058       5        1  11.284717
4      0.172047 -0.159344       6        1  11.049140
2      0.123808  0.152483       7        1  10.974693
7     -0.024815 -0.187724       8        1   7.981898, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
78    Default   775.000000         xbox   775.000000  30.0000  30.0000
29    Default  1048.000000       window  1048.000000  29.0000  29.0000
311   Default   623.000000         game   623.000000  28.0000  28.0000
222   Default   434.000000      android   434.000000  27.0000  27.0000
258   Default   589.000000       market   589.000000  26.0000  26.0000
279   Default   483.000000           pc   483.000000  25.0000  25.0000
75    Default   349.000000          one   349.000000  24.0000  24.0000
298   Default   301.000000         halo   301.000000  23.0000  23.0000
241   Default   304.000000        march   304.000000  22.0000  22.0000
97    Default   384.000000         data   384.000000  21.0000  21.0000
266   Default   284.000000         live   284.000000  20.0000  20.0000
392   Default   263.000000      project   263.000000  19.0000  19.0000
10    Default   420.000000       update   420.000000  18.0000  18.0000
264   Default   246.000000        award   246.000000  17.0000  17.0000
15    Default   260.000000         news   260.000000  16.0000  16.0000
267   Default   217.000000        music   217.000000  15.0000  15.0000
265   Default   214.000000  iheartradio   214.000000  14.0000  14.0000
567   Default   207.000000   collection   207.000000  13.0000  13.0000
566   Default   202.000000        chief   202.000000  12.0000  12.0000
569   Default   197.000000       master   197.000000  11.0000  11.0000
1098  Default   194.000000       xcloud   194.000000  10.0000  10.0000
87    Default   324.000000        cloud   324.000000   9.0000   9.0000
562   Default   200.000000           io   200.000000   8.0000   8.0000
16    Default   221.000000       office   221.000000   7.0000   7.0000
481   Default   335.000000      service   335.000000   6.0000   6.0000
837   Default   203.000000       coming   203.000000   5.0000   5.0000
73    Default   182.000000           gb   182.000000   4.0000   4.0000
34    Default   252.000000          app   252.000000   3.0000   3.0000
188   Default   236.000000          key   236.000000   2.0000   2.0000
190   Default   234.000000       player   234.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
605    Topic8    63.583015         work    64.434113   2.5147  -4.2167
448    Topic8    61.876293     internet    62.727306   2.5143  -4.2439
247    Topic8    55.140228       device    55.991554   2.5127  -4.3591
413    Topic8    54.633915          pay    55.484684   2.5125  -4.3684
674    Topic8    52.399719       reject    53.250446   2.5119  -4.4101
442    Topic8    51.721897         word    52.572884   2.5117  -4.4231
310    Topic8    51.037083         play    51.888306   2.5115  -4.4365
537    Topic8    48.381363           th    49.232483   2.5106  -4.4899
673    Topic8    46.761162        never    47.611877   2.5100  -4.5240
415    Topic8    45.223774         look    46.075008   2.5093  -4.5574
151    Topic8    44.748013      limited    45.599434   2.5091  -4.5680
89     Topic8    44.693409         code    45.544510   2.5091  -4.5692
748    Topic8    44.241764       camera    45.093010   2.5089  -4.5794
83     Topic8    43.248669   specialist    44.099636   2.5085  -4.6021
148    Topic8    43.009125     explorer    43.860119   2.5084  -4.6076
245    To

In [210]:
def vec_similarity(input1, input2, vectors):
    term_vectors = [np.zeros(300), np.zeros(300)]
    terms = [input1, input2]
        
    for index, term in enumerate(terms):
        for i, t in enumerate(term.split(' ')):
            try:
                term_vectors[index] += vectors[t]
            except:
                term_vectors[index] += 0
        
    result = (1 - spatial.distance.cosine(term_vectors[0], term_vectors[1]))
    if result is 'nan':
        result = 0
        
    return result

# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

# function calculates similarity between two strings using a particular word vector model
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    if len(s1words) == 0 or len(s2words) == 0:
        return 0 
    output = vectors.n_similarity(s1words, s2words)
    return output

In [211]:
def vec_similarity(input1, input2, vectors):
    term_vectors = [np.zeros(300), np.zeros(300)]
    terms = [input1, input2]
        
    for index, term in enumerate(terms):
        for i, t in enumerate(term.split(' ')):
            try:
                term_vectors[index] += vectors[t]
            except:
                term_vectors[index] += 0
        
    result = (1 - spatial.distance.cosine(term_vectors[0], term_vectors[1]))
    if result is 'nan':
        result = 0
        
    return result

In [212]:
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [213]:
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    output = vectors.n_similarity(s1words, s2words)
    return output

In [214]:
def cleanup(input):
    # remove English stopwords
    input = input.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
    input = re.sub(r'[^a-zA-Z0-9 ]', '', input)
    return input

In [215]:
from simhash import Simhash, SimhashIndex

In [216]:
distance = 25
objs = [(str(feed['id']), Simhash(str(feed['title']))) for feed in feeds]
index = SimhashIndex(objs, k=distance)

In [46]:
# function takes an input string, runs similarity for each item in topic_taxonomy, sorts and returns top 3 results
def classify_topics(input, vectors):
    feed_score = dict()
    for key, value in topic_taxonomy.items():
        max_value_score = dict()
        for label, keywords in value.items():
            max_value_score[label] = 0
            topic = (key + ' ' + keywords).strip()
            max_value_score[label] += float(calc_similarity(input, topic, vectors))
            
        sorted_max_score = sorted(max_value_score.items(), key=operator.itemgetter(1), reverse=True)[0]
        feed_score[sorted_max_score[0]] = sorted_max_score[1]
    return sorted(feed_score.items(), key=operator.itemgetter(1), reverse=True)[:3]

In [48]:
#model_fasttext = load_wordvec_model('FastText', 'fastText_wiki_en.vec', False)